# Fundamentos do desenvolvimento de modelos analíticos em Python

Ficha de trabalho 03

A **Regressão Logística** é um modelo de **classificação binária**, que prevê a probabilidade de um resultado entre duas classes (ex: sobreviver ou não).
Apesar do nome, **não serve para prever valores contínuos**. A saída é uma **probabilidade** entre 0 e 1, convertida depois numa classe 0 ou 1.

A regressão logística usa a função **sigmóide**:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

A função Regressão Logística é usada porque:
* É simples e eficaz para introdução ao ML;
* Ideal para classificação binária;
* F*unciona bem com variáveis numéricas ou categóricas codificadas;
* Interpretável: permite perceber o peso de cada variável.

In [6]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

* pandas: biblioteca para manipulação de dados tabulares (DataFrames)
* train_test_split: separa os dados em conjuntos de treino e teste
* LogisticRegression: classe que define o modelo de regressão logística
* accuracy_score, confusion_matrix, classification_report: métricas para avaliação do modelo#

# Carregar e preparar o dataset

In [ ]:
import os

source_filename = 'data/titanic.csv'
source_online = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
if not os.path.exists(source_filename):
    pd.read_csv(source_online).to_csv(source_filename, index=False)
    print('download titanic dataset from online source')

df = pd.read_csv(source_filename)
print('carregado dataset titanic')

carregado dataset titanic


In [8]:
# Seleciona apenas as colunas relevantes e remove linhas com valores em falta
df = df[['Survived', 'Pclass', 'Sex', 'Age']].dropna()

In [9]:
# Converte os valores categóricos em numéricos
df['Sex'] = df['Sex'].map(
    {
        'male': 0, 
        'female': 1
    }
)

* selecionam-se apenas as colunas relevantes
* dropna(): remove linhas com valores em falta
* map(...): converte "male"/"female" para 0/1 (necessário para ML)

# Definição das variáveis independestes e dependentes

In [10]:
X = df[['Pclass', 'Sex', 'Age']]
y = df['Survived']

X_trian, X_text, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

* X e y são definidos separadamente
* train_test_split: 70% treino e 30% teste, com semente (random_state) para repetibilidade

In [11]:
X_trian.shape, X_text.shape, y_train.shape, y_test.shape

((535, 3), (179, 3), (535,), (179,))

# Definição do modelo

In [13]:
# criação do modelo com número máximo de iterações fixado
model = LogisticRegression(max_iter = 200)

# treino do modelo com os dados de treino
model.fit(X_trian, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,200
,multi_class,'deprecated'


* Criação do modelo com número máximo de iterações fixado
* fit(...): treino do modelo com dados de treino

# Previsão

In [14]:
y_pred = model.predict(X_text)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.770949720670391
Matriz de Confusão:
 [[87 20]
 [21 51]]
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.81      0.81      0.81       107
           1       0.72      0.71      0.71        72

    accuracy                           0.77       179
   macro avg       0.76      0.76      0.76       179
weighted avg       0.77      0.77      0.77       179



* predict(...): previsões com os dados de teste
* accuracy_score: percentagem de classificações corretas
* confusion_matrix: matriz com TP, TN, FP, FN
* classification_report: métricas detalhadas como precisão, revocação e F1-score


A função confusion_matrix do scikit-learn calcula uma matriz de confusão, que é uma tabela que permite avaliar o desempenho de um modelo de classificação, comparando os valores reais com os valores previstos.
* TP (True Positive): O modelo previu 1 e o rótulo real era 1 → previsão correta de positivo.
* TN (True Negative): O modelo previu 0 e o rótulo real era 0 → previsão correta de negativo.
* FP (False Positive): O modelo previu 1 mas o rótulo real era 0 → "falso alarme".
* FN (False Negative): O modelo previu 0 mas o rótulo real era 1 → "falha na deteção".


# Interpretação dos coeficientes

In [15]:
for nome, coef in zip(X.columns, model.coef_[0]):
    print(f'Coeficiente de {nome}: {coef:.4f}')

Coeficiente de Pclass: -1.2385
Coeficiente de Sex: 2.4623
Coeficiente de Age: -0.0425


* Mostra o peso de cada variável no modelo
* Coeficiente positivo → aumenta probabilidade de classe 1 (sobreviveu)
* Coeficiente negativo → diminui probabilidade

# Overfitting e Validação Cruzada

In [16]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=200)
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy') # Divide o treino em 5 partes e avalia a estabilidade do modelo

print('Scores:', scores)
print('Média da acurácia:', scores.mean())

Scores: [0.76923077 0.83216783 0.77622378 0.74825175 0.8028169 ]
Média da acurácia: 0.7857382054565154


**Overfitting**
* Modelo aprende demais os dados de treino → falha em dados novos

**Estratégias para evitar**:
* Separar treino/teste
* Validação cruzada
* Regularização

# Atividade Resolvida: Simular overtitting

## Exemplo 1: treino com 5 exemplos

In [17]:
amostra = df.sample(5, random_state=42)
X_amostra = amostra[['Pclass', 'Sex', 'Age']]
y_amostra = amostra['Survived']

modelo_pequeno = LogisticRegression()
modelo_pequeno.fit(X_amostra, y_amostra)

print('Acurácia terino (5 exemplos):', accuracy_score(y_amostra, modelo_pequeno.predict(X_amostra)))

modelo_pequeno.predict(X_amostra)

Acurácia terino (5 exemplos): 1.0


array([0, 1, 1, 1, 0])

* O modelo "memoriza" → overfitting

## Exemplo 2: Adicionar ruído

In [18]:
import numpy as np

df_100 = df.sample(n=100, random_state=42)

X_100 = df_100[['Pclass', 'Sex', 'Age']].copy()

for i in range(5):
    X_100[f'ruido_{i}'] = np.random.rand(len(X_100))

y_100 = df_100['Survived']

modelo_ruido = LogisticRegression()
modelo_ruido.fit(X_100, y_100)
print('Acurácia com 100 exemplos e ruído:', accuracy_score(y_100, modelo_ruido.predict(X_100)))

Acurácia com 100 exemplos e ruído: 0.78


* O modelo ajusta-se a variáveis sem sentido

## Exemplo 3: Aplicar regularização

In [19]:
modelo_reg = LogisticRegression(C=0.01)
modelo_reg.fit(X_100, y_100)

print('Acurácia com regularização:', accuracy_score(y_100, modelo_reg.predict(X_100)))

for nome, coef in zip(X_100.columns, modelo_reg.coef_[0]):
    print(f'Coeficiente de {nome}: {coef:.4f}')

Acurácia com regularização: 0.64
Coeficiente de Pclass: -0.1512
Coeficiente de Sex: 0.0728
Coeficiente de Age: 0.0143
Coeficiente de ruido_0: 0.0249
Coeficiente de ruido_1: -0.0035
Coeficiente de ruido_2: 0.0056
Coeficiente de ruido_3: -0.0068
Coeficiente de ruido_4: -0.0072


* Coeficientes de variáveis irrelevantes tendem a 0 → modelo mais robusto

***